In [21]:
import numpy as np
from scipy import stats
import math
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import pandas as pd
import geopandas as gpd

In [22]:
def custom_dist(u, v):
    if u[2] == v[2]:
        return sys.float_info.max
    else:
        return haversine([u[0], u[1]], [v[0], v[1]])

thresholds = {'CurbRamp': 0.0075,
                      'NoCurbRamp': 0.0075,
                      'SurfaceProblem': 0.01,
                      'Obstacle': 0.01,
                      'NoSidewalk': 0.01,
                      'Crosswalk': 0.01,
                      'Signal': 0.01,
                      'Occlusion': 0.01,
                      'Other': 0.01,
                      'Problem': 0.01}

def cluster(labels, label_type, thresholds):

    # Makes a normal dist matrix for a single user, but uses special dist function for multi-user clustering that
    # prevents the same user's attributes from being clustered together.
    dist_matrix = pdist(np.array(labels[['lat', 'lng', 'user_id']].values), custom_dist)
    link = linkage(dist_matrix, method='complete')
    curr_type = labels['label_type'].values

    # Copies the labels dataframe and adds a column to it for the cluster id each label is in.
    labelsCopy = labels.copy()
    labelsCopy.loc[:,'cluster_id'] = fcluster(link, t=thresholds[label_type], criterion='distance')

    # Cuts tree so that only labels less than clust_threth kilometers apart are clustered.
    clusters = labelsCopy.groupby('cluster_id')

    # Computes the center of each cluster and assigns temporariness and severity.
    cluster_list = [] # list of tuples (label_type, cluster_num, lat, lng, severity, temporary).
    for clust_num, clust in clusters:
        #ave_pos = np.mean(clust['coords'].tolist(), axis=0) # use ave pos of clusters.
        #ave_sev = None if pd.isnull(clust['severity']).all() else int(round(np.median(clust['severity'][~np.isnan(clust['severity'])])))
        #ave_temp = None if pd.isnull(clust['temporary']).all() else bool(round(np.mean(clust['temporary'])))

        cluster_list.append((curr_type, clust_num))

    cluster_df = pd.DataFrame(cluster_list, columns=['label_type', 'cluster_id'])

    return (cluster_df, labelsCopy)


# Pick which label types should be included in clustering, and which should be included in the "Problem" type.
label_types = ['CurbRamp', 'NoSidewalk', 'Occlusion', 'SurfaceProblem', 'Obstacle', 'Other', 'NoCurbRamp', 'Crosswalk', 'Signal']
# problem_types = ['SurfaceProblem', 'Obstacle']

# These are the columns required in the POST requests for the labels and clusters, respectively.
label_cols = ['label_id', 'label_type', 'cluster_id']
cluster_cols = ['label_type', 'cluster_id', 'lat', 'lng', 'severity', 'temporary']


def cluster_label_type_at_index(label_data,i):
        clusters_for_type_i = pd.DataFrame(columns=cluster_cols)
        labels_for_type_i = pd.DataFrame(columns=label_cols)

        label_type = label_types[i]
        # if label_type == 'Problem':
        #     type_data = label_data[label_data.label_type.isin(problem_types)]
        # else:
        type_data = label_data[label_data.label_type == label_type]

        # If there are >1 labels, we can do clustering. Otherwise just copy the 1 (or 0) labels.
        if type_data.shape[0] > 1:
            (clusters_for_type_i, labels_for_type_i) = cluster(type_data, label_type, thresholds)
        elif type_data.shape[0] == 1:
            labels_for_type_i = type_data.copy()
            labels_for_type_i.loc[:,'cluster_id'] = 1 # Gives the single cluster a cluster_id of 1.
            labels_for_type_i.loc[:,'label_type'] = label_type # Gives Problem type if needed.
            clusters_for_type_i = labels_for_type_i.filter(items=cluster_cols)

        return (label_type, clusters_for_type_i, labels_for_type_i)

In [23]:
df = gpd.read_file('data/processed-labels/q1.shp')

In [24]:
df

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry
0,85242,CurbRamp,3.0,ta67csM3WvZLCgxCYaiYXg,1,8.482142,-24.339285,77.655418,2.281700,e60908c6-24b9-44d9-9561-8612f819b709,47.656212,-122.385529,POINT (-122.38553 47.65621)
1,85334,CurbRamp,1.0,_KAca6iODLmELefF17KMyg,1,277.250000,-3.687500,89.608833,-2.126404,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.656628,-122.349808,POINT (-122.34981 47.65663)
2,85336,CurbRamp,1.0,_KAca6iODLmELefF17KMyg,1,277.250000,-3.687500,89.608833,-2.126404,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.656635,-122.349815,POINT (-122.34982 47.65664)
3,85345,CurbRamp,1.0,I0lp4iL7RsC-ReloBy7dEA,1,304.062500,-23.812500,176.969223,3.324486,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.657665,-122.350105,POINT (-122.35011 47.65767)
4,85350,SurfaceProblem,3.0,94k4ZjWrmFyO6dLOt0Rong,1,316.250000,-15.656250,90.449310,5.476624,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.657784,-122.350525,POINT (-122.35052 47.65778)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55796,230580,NoCurbRamp,2.0,KmeI1UmNtZ4Yb8p1xoD05A,2,96.035713,-20.267857,5.991113,-0.645590,78da7c35-4512-49df-a8f3-d5142b71df03,47.693096,-122.325638,POINT (-122.32564 47.69310)
55797,230950,NoSidewalk,3.0,UbUOb8U2HoONf77QS2E5pw,1,138.125000,-18.062500,90.515388,1.982470,a1dcc246-35ef-493f-ab99-80afa75169a6,47.705021,-122.336075,POINT (-122.33607 47.70502)
55798,230954,NoSidewalk,4.0,vbUhTvBpS2cPPxij2o5oVg,2,258.142853,-5.446429,257.855042,-6.324650,a1dcc246-35ef-493f-ab99-80afa75169a6,47.703163,-122.337822,POINT (-122.33782 47.70316)
55799,230964,NoCurbRamp,4.0,isk8T0jXzG1NAJu_bOdaCg,3,293.687500,-6.578125,289.504578,-2.199040,a1dcc246-35ef-493f-ab99-80afa75169a6,47.692432,-122.340370,POINT (-122.34037 47.69243)


In [25]:
df0 = cluster_label_type_at_index(df,0)[2]

In [26]:
df1 = cluster_label_type_at_index(df,1)[2]

In [27]:
df2 = cluster_label_type_at_index(df,2)[2]

In [28]:
df3 = cluster_label_type_at_index(df,3)[2]

In [29]:
df4 = cluster_label_type_at_index(df,4)[2]

In [30]:
df5 = cluster_label_type_at_index(df,5)[2]

In [31]:
df6 = cluster_label_type_at_index(df,6)[2]

In [32]:
df7 = cluster_label_type_at_index(df,7)[2]

In [33]:
df8 = cluster_label_type_at_index(df,8)[2]

In [34]:
df0["clustered"] = df0.duplicated(subset="cluster_id", keep=False)
df1["clustered"] = df1.duplicated(subset="cluster_id", keep=False)
df2["clustered"] = df2.duplicated(subset="cluster_id", keep=False)
df3["clustered"] = df3.duplicated(subset="cluster_id", keep=False)
df4["clustered"] = df4.duplicated(subset="cluster_id", keep=False)
df5["clustered"] = df5.duplicated(subset="cluster_id", keep=False)
df6["clustered"] = df6.duplicated(subset="cluster_id", keep=False)
df7["clustered"] = df7.duplicated(subset="cluster_id", keep=False)
df8["clustered"] = df8.duplicated(subset="cluster_id", keep=False)

In [35]:
df0

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry,cluster_id,clustered
0,85242,CurbRamp,3.0,ta67csM3WvZLCgxCYaiYXg,1,8.482142,-24.339285,77.655418,2.281700,e60908c6-24b9-44d9-9561-8612f819b709,47.656212,-122.385529,POINT (-122.38553 47.65621),9897,True
1,85334,CurbRamp,1.0,_KAca6iODLmELefF17KMyg,1,277.250000,-3.687500,89.608833,-2.126404,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.656628,-122.349808,POINT (-122.34981 47.65663),10653,True
2,85336,CurbRamp,1.0,_KAca6iODLmELefF17KMyg,1,277.250000,-3.687500,89.608833,-2.126404,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.656635,-122.349815,POINT (-122.34982 47.65664),7667,False
3,85345,CurbRamp,1.0,I0lp4iL7RsC-ReloBy7dEA,1,304.062500,-23.812500,176.969223,3.324486,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.657665,-122.350105,POINT (-122.35011 47.65767),10682,True
6,85359,CurbRamp,1.0,2RgFrskrqXsgS48Dy9Xc4g,1,306.765625,-23.562500,180.722519,1.948097,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.655487,-122.351234,POINT (-122.35123 47.65549),7517,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55772,84509,CurbRamp,1.0,pDHV1I3r_ET-mV4vwcMTFA,3,333.627228,-7.968750,179.299835,-0.525291,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.697865,-122.335243,POINT (-122.33524 47.69786),28,True
55784,230432,CurbRamp,2.0,lLOTBFQ82wKFEVpK2Cuuuw,2,150.267853,-12.428572,356.792969,-0.589950,78da7c35-4512-49df-a8f3-d5142b71df03,47.694984,-122.328323,POINT (-122.32832 47.69498),24,True
55787,230460,CurbRamp,1.0,kaYOm6_wYw9GASKK97QcBg,2,133.571426,-28.089285,18.108595,-0.323940,78da7c35-4512-49df-a8f3-d5142b71df03,47.690380,-122.324303,POINT (-122.32430 47.69038),19,True
55792,230439,CurbRamp,2.0,87un5olifsw0VKGBL4sEFA,2,7.678571,-16.410715,21.724104,-1.176300,78da7c35-4512-49df-a8f3-d5142b71df03,47.695061,-122.326996,POINT (-122.32700 47.69506),8,True


In [36]:
#concatenate all dataframes
clusters_q1 = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8])

In [37]:
clusters_q1

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry,cluster_id,clustered
0,85242,CurbRamp,3.0,ta67csM3WvZLCgxCYaiYXg,1,8.482142,-24.339285,77.655418,2.281700,e60908c6-24b9-44d9-9561-8612f819b709,47.656212,-122.385529,POINT (-122.38553 47.65621),9897,True
1,85334,CurbRamp,1.0,_KAca6iODLmELefF17KMyg,1,277.250000,-3.687500,89.608833,-2.126404,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.656628,-122.349808,POINT (-122.34981 47.65663),10653,True
2,85336,CurbRamp,1.0,_KAca6iODLmELefF17KMyg,1,277.250000,-3.687500,89.608833,-2.126404,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.656635,-122.349815,POINT (-122.34982 47.65664),7667,False
3,85345,CurbRamp,1.0,I0lp4iL7RsC-ReloBy7dEA,1,304.062500,-23.812500,176.969223,3.324486,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.657665,-122.350105,POINT (-122.35011 47.65767),10682,True
6,85359,CurbRamp,1.0,2RgFrskrqXsgS48Dy9Xc4g,1,306.765625,-23.562500,180.722519,1.948097,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.655487,-122.351234,POINT (-122.35123 47.65549),7517,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49369,222490,Signal,NaN,-dXs1_aXe4i9N2BvuQu2cg,3,217.555801,-10.890625,270.065277,3.653336,6b5dd352-f2a7-4274-ba26-54717e5410b5,47.695869,-122.334877,POINT (-122.33488 47.69587),12,False
49860,231077,Signal,NaN,PlClKXA8ouVEUv33dO0DAA,1,6.000000,-10.000000,233.264282,-0.090750,28c0b06a-5d3f-4fb8-8341-575c1859f08a,47.696491,-122.372635,POINT (-122.37263 47.69649),10,False
49861,231079,Signal,NaN,PlClKXA8ouVEUv33dO0DAA,1,6.000000,-10.000000,233.264282,-0.090750,28c0b06a-5d3f-4fb8-8341-575c1859f08a,47.696510,-122.372673,POINT (-122.37267 47.69651),8,False
49862,231080,Signal,NaN,PlClKXA8ouVEUv33dO0DAA,1,6.000000,-10.000000,233.264282,-0.090750,28c0b06a-5d3f-4fb8-8341-575c1859f08a,47.696548,-122.372681,POINT (-122.37268 47.69655),5,False


In [38]:
#export to shapefile
clusters_q1.to_file("data/processed-labels/q1_clusters.shp")